In [ ]:
# default_exp ads

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
!pip install -qU nbdev fastcore

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 40kB 3.9MB/s 


In [ ]:
%%capture
#hide
!pip install -qU gspread facebook_business python-facebook-api

In [ ]:
#hide
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


In [ ]:
#hide
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

## Facebook Auth

In [ ]:
%%capture
#hide

import yaml
with open("facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env AD_ACC_ID {data.get("AD_ACC_ID")}
    %env USER_ID {data.get("USER_ID")}
    %env AWS_ACCESS_KEY_ID {data.get("AWS_ACCESS_KEY_ID")}
    %env AWS_SECRET_ACCESS_KEY {data.get("AWS_SECRET_ACCESS_KEY")}
    %env BUCKET {data.get("BUCKET")}

In [ ]:
#export
from tomorrowswig_followers.core import *

In [ ]:
#export
import os
from typing import *

from datetime import datetime, timedelta
from functools import partial

import pandas as pd
import numpy as np
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adaccount import AdAccount

AD_ACC_ID = os.environ.get("AD_ACC_ID")

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Ads reporting

In [ ]:
#export
FacebookAdsApi.init(APP_ID, APP_SECRET, TOKEN)
account = AdAccount(AD_ACC_ID)

In [ ]:
# account.get_campaigns(fields=["status", "effective_status", "name",], params = {
#   'effective_status': ['ACTIVE'], "is_completed": False,
# })

In [ ]:
# account.get_ad_sets(fields=["status", "effective_status", "name", "learning_stage_info"], )

https://github.com/facebook/facebook-python-business-sdk/blob/master/facebook_business/adobjects/adsinsights.py

In [ ]:
insights = account.get_insights(fields=[
    "ad_id",
    "adset_id",
    "ad_name",
    "clicks",
    "cpc",
    "ctr",
    "cpm",
    "cost_per_action_type",
    "spend",
    "impressions",
    "reach",
    "actions",
    "video_avg_time_watched_actions",
    "video_p50_watched_actions",
    "video_p75_watched_actions",
    "video_p95_watched_actions",

], params={
    'level': "ad",
    'date_preset': "yesterday",
    # "date_start": "1595116800",
    # "date_stop": "1595203200"
})


In [ ]:
from facebook_business.adobjects.adsinsights import *
fields = [f for f in AdsInsights.Field.__dict__ if not f.startswith("__")]

In [ ]:
" ".join([fields[56], fields[105]])

'cost_per_unique_conversion unique_conversions'

In [ ]:
# account.get_insights(fields=["ad_id", "quality_ranking", "engagement_rate_ranking",
#                              "conversion_rate_ranking", "ad_name"], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.yesterday,
# })

In [ ]:
#export
COLUMNS = ["Ad Name", "Ads Followers Change", "% (Clicks)", "% (Impressions)", "Cost Per Follow", "Delivery Type",
           "Clicks (All)", "Link Clicks", "CPC (All)" , "CPC (Cost per Link Click)",
           "CTR (All)", "CPM (Cost per 1,000 Impressions)", "Amount Spent (USD)", "Impressions", "Reach", "Post Reactions", "Post Shares",
           "Video Average Play Time", "Video Plays at 50%", "Video Plays at 75%", "Video Plays at 95%"]

In [ ]:
#export
def get_insights(date: str) -> Dict:
    return account.get_insights(fields=[
        "ad_id",
        "adset_id",
        "ad_name",
        "clicks",
        "cpc",
        "ctr",
        "cpm",
        "cost_per_action_type",
        "spend",
        "impressions",
        "reach",
        "actions",
        "video_avg_time_watched_actions",
        "video_p50_watched_actions",
        "video_p75_watched_actions",
        "video_p95_watched_actions",
    ], params={
        "level": "ad",
        "time_range": {"since": date,
                       "until": date},
    })

In [ ]:
#export
def get_action(cell, name):
    if isinstance(cell, float):
        return 0
    action = [a for a in cell if a["action_type"] == name]
    if action:
        return float(action[0]["value"])
    return 0

In [ ]:
#export
def add_country_total(df: pd.DataFrame) -> None:
    dupes = df[df.index.duplicated(False)]
    if not len(dupes):
        return

    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)"]
    to_sum = set(df.columns).difference(set(to_mean + ["ad_name", "Delivery Type"]))
    total = dupes.iloc[0,:].copy()
    index = dupes.index[0]
    total["ad_name"] = f"{index} total"
    total[to_mean] = dupes[to_mean].apply(np.mean)
    total[to_sum] = dupes[to_sum].astype(int).apply(sum)
    df = df.append(total)

In [ ]:
ad = Ad("23844751184140691")

In [ ]:
ad = ad.api_get(fields=["targeting"])
ad["targeting"]["geo_locations"]["countries"][0]

'MX'

In [ ]:
AdSet("23844063827180691").api_get(fields=["learning_stage_info", "status"])

<AdSet> {
    "id": "23844063827180691",
    "status": "ACTIVE"
}

In [ ]:
#export
def get_delivery(ids: List[str]) -> List[Dict]:
    deliveries = []
    for id in ids:
        adset = AdSet(id[0]).api_get(fields=["learning_stage_info", "status"])
        delivery = adset.get("learning_stage_info")
        status = adset.get("status")
        if delivery:
            delivery = status if delivery.get("status") == "SUCCESS" else delivery.get("status")
        else:
            delivery = status
        deliveries.append({"ad_id": id[1], "adset_id": id[0], "Delivery Type": delivery})
    return deliveries

In [ ]:
assert get_delivery([["23844128955750691", "23844703941280691"],
                     ["23844223938640691","23844703855040691"],
                     ["23844063827180691", "23844751253080691"]]) == [{'Delivery Type': 'ACTIVE',
                                                                       'ad_id': '23844703941280691',
                                                                       'adset_id': '23844128955750691'},
                                                                      {'Delivery Type': 'PAUSED',
                                                                       'ad_id': '23844703855040691',
                                                                       'adset_id': '23844223938640691'},
                                                                      {'Delivery Type': 'ACTIVE',
                                                                       'ad_id': '23844751253080691',
                                                                       'adset_id': '23844063827180691'}]

In [ ]:
#export
def get_countries() -> pd.DataFrame:
    """Get countries of all ads"""
    countries = []
    ads = account.get_ads(fields=["id", "targeting"])
    for ad in ads:
        # ad = Ad(id).api_get(fields=["targeting"])
        locs = ad["targeting"].get("geo_locations")
        if locs:
            c = locs.get("countries")
            if c:
                country = c[0]
            else:
                country = "US"
            countries.append({"ad_id": ad["id"], "country": country})
    return pd.DataFrame(countries)

In [ ]:
#export
def add_total(df: pd.DataFrame) -> pd.DataFrame:
    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time"]
    empty = ["Ads Followers Change", "% (Clicks)", "% (Impressions)", "Cost Per Follow", "ad_name", "Delivery Type"]
    to_sum = df.columns.difference(to_mean + empty)
    total = pd.Series(name="total", index=df.columns, dtype=float)
    total[to_mean] = df[to_mean].apply(np.mean)
    total[to_sum] = df[to_sum].apply(sum)
    df = df.append(total)
    return pd.concat([df.iloc[-1:], df.iloc[:-1]])

In [ ]:
#export
def get_insights_df(countries: pd.DataFrame, insights: List) -> pd.DataFrame:
    df = pd.DataFrame(insights)
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)

    df = df.merge(countries)
    df = df.merge(pd.DataFrame(get_delivery(df[["adset_id", "ad_id"]].values)))
    df.set_index(df["country"], inplace=True)
    df.sort_values("ad_name", inplace=True)

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click"))
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions", "ad_id", "adset_id", "country"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    add_country_total(df)
    
    df = add_total(df)
    df.rename(columns={"ad_name": "Ad Name", "clicks": "Clicks (All)", "cpc": "CPC (All)",
                       "ctr": "CTR (All)", "cpm": "CPM (Cost per 1,000 Impressions)",
                       "reach": "Reach", "impressions": "Impressions", "spend": "Amount Spent (USD)"}, inplace=True)
    df["Ads Followers Change"] = 0
    df["% (Clicks)"] = 0
    df["% (Impressions)"] = 0
    df["Cost Per Follow"] = 0
    return df[COLUMNS].round(2)

In [ ]:
#export
def create_insights(event: Dict = None, context=None,) -> str:
    yesterday = datetime.today() - timedelta(days=1)
    insights = get_insights(yesterday.strftime("%Y-%m-%d"))
    if not insights:
        return f"No insights found for {yesterday.strftime('%Y-%m-%d')}"
    countries = get_countries()
    df = get_insights_df(countries, insights)
    df_size = len(df)

    empty = pd.Series(name="", dtype=str)
    df = df.append([empty] * 25)

    worksheet_name = yesterday.strftime("%b %d %Y")
    write_df(df, worksheet_name, worksheet_loc=8)
    notes_df = pd.DataFrame([["", "CONSIDER:", "", "", "TO DO:"]], index=["PERFORMANCE:"], columns=[""] * 5)
    write_df(notes_df, worksheet_name, loc=f"A{df_size+10}")

    wsh = get_worksheet(worksheet_name)
    wsh.format(f"C1:F{df_size+1}", {"textFormat": {
        "foregroundColor": {
            "red": 0.45,
            "green": 0.0,
            "blue": 0.0
        }}},)
    wsh.format(f"D:E", {"numberFormat": {"type": "PERCENT"}},)
    wsh.format(f"F", {"numberFormat": {"type": "CURRENCY"}},)
    return f"Created insights in '{worksheet_name}'"

In [ ]:
create_insights()

'No insights found for 2020-09-27'

In [ ]:
#export
def add_followers(df: pd.DataFrame, new_followers: pd.DataFrame):
    df["Ads Followers Change"] = new_followers
    df.fillna(0, inplace=True)
    df["% (Clicks)"] = (df["Ads Followers Change"] / df["Clicks (All)"])
    df["% (Impressions)"] = (df["Ads Followers Change"] / df["Impressions"])
    df["Cost Per Follow"] = (df["Amount Spent (USD)"] / df["Ads Followers Change"])
    df.replace([np.inf], 0, inplace=True)
    df["% (Impressions)"] = df["% (Impressions)"].round(4)
    df[df.columns.difference(["% (Impressions)"])] = df[df.columns.difference(["% (Impressions)"])].round(2)

In [ ]:
#export
def get_insights_for(date: str, countries: pd.DataFrame, followers_change: pd.DataFrame) -> pd.DataFrame:
    iso_date = datetime.strptime(date, "%b %d %Y").strftime("%Y-%m-%d")
    insights = get_insights(iso_date)
    if not insights:
        return pd.DataFrame()
    df = get_insights_df(countries, insights).drop("total")
    add_followers(df, followers_change)
    df["Ads Followers Change"] = followers_change
    df["Date"] = iso_date
    df.reset_index(inplace=True)
    df.set_index(df["Date"], inplace=True)
    return df.drop(columns=["Date"])

In [ ]:
#export
def update_dashboard(date: str, followers_change: pd.DataFrame):
    sheet = "Dashboard - Ad Data"
    df = get_df(sheet)
    countries = get_countries()
    insights_df = get_insights_for(date, countries, followers_change)
    write_df(pd.concat([insights_df, df]), sheet)
    return f"Updated {sheet} with {date} data"

In [ ]:
countries = get_countries()
followers_change = pd.DataFrame({"2020-07-21": {'AR': 100, 'AU': 70, 'BD': 0, 'BR': 0, 'CA': 0, 'CL': 0, 'CO': 0}})
get_insights_for("Jul 21 2020", countries, followers_change)

,country,Ad Name,Ads Followers Change,% (Clicks),% (Impressions),Cost Per Follow,Delivery Type,Clicks (All),Link Clicks,CPC (All),CPC (Cost per Link Click),CTR (All),"CPM (Cost per 1,000 Impressions)",Amount Spent (USD),Impressions,Reach,Post Reactions,Post Shares,Video Average Play Time,Video Plays at 50%,Video Plays at 75%,Video Plays at 95%
Date,,,,,,,,,,,,,,,,,,,,,,
2020-07-21,MX,2 - Mexico - General Ad #2-D,NaN,0.0,0.0000,0.00,ACTIVE,75.0,36.0,0.04,0.08,1.12,0.44,2.92,6689.0,6584.0,2.0,0.0,2.0,1146.0,283.0,172.0
2020-07-21,AR,3 - Argentina - General 2-D,100.0,2.0,0.0116,0.03,PAUSED,50.0,25.0,0.06,0.12,0.58,0.34,2.96,8587.0,8587.0,0.0,1.0,1.0,1205.0,250.0,168.0
2020-07-21,ID,3 - Indonesia - General Ad 2-D,NaN,0.0,0.0000,0.00,ACTIVE,132.0,59.0,0.03,0.08,0.96,0.33,4.52,13740.0,13620.0,4.0,1.0,2.0,2060.0,645.0,409.0


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
